In [484]:
using ITensors
using Random, Distributions

# Alternative Overlap Attempt

In [485]:
sites = siteinds("S=1/2", 5);

In [486]:
Random.seed!(42);
mps = randomMPS(sites; linkdims=2);

In [487]:
product_state = randomMPS(sites; linkdims=1)

MPS
[1] ((dim=2|id=540|"S=1/2,Site,n=1"), (dim=1|id=955|"Link,l=1"))
[2] ((dim=1|id=955|"Link,l=1"), (dim=2|id=615|"S=1/2,Site,n=2"), (dim=1|id=601|"Link,l=2"))
[3] ((dim=1|id=601|"Link,l=2"), (dim=2|id=927|"S=1/2,Site,n=3"), (dim=1|id=215|"Link,l=3"))
[4] ((dim=1|id=215|"Link,l=3"), (dim=2|id=814|"S=1/2,Site,n=4"), (dim=1|id=523|"Link,l=4"))
[5] ((dim=1|id=523|"Link,l=4"), (dim=2|id=945|"S=1/2,Site,n=5"))


In [369]:
inner(product_state, mps)

0.3197566132127257

Now that we have the ground truth, let's construct an effective overlap. To make sure each step is being done properly, we will do it in parts.

# Site 1

In [371]:
mps_ket = mps
mps_bra = dag(prime(mps))

MPS
[1] ((dim=2|id=139|"S=1/2,Site,n=1")', (dim=2|id=316|"Link,l=1")')
[2] ((dim=2|id=316|"Link,l=1")', (dim=2|id=902|"S=1/2,Site,n=2")', (dim=2|id=846|"Link,l=2")')
[3] ((dim=2|id=846|"Link,l=2")', (dim=2|id=593|"S=1/2,Site,n=3")', (dim=2|id=127|"Link,l=3")')
[4] ((dim=2|id=127|"Link,l=3")', (dim=2|id=87|"S=1/2,Site,n=4")', (dim=2|id=179|"Link,l=4")')
[5] ((dim=2|id=179|"Link,l=4")', (dim=2|id=443|"S=1/2,Site,n=5")')


In [372]:
phi_1_ket = product_state[1] * mps_ket[1] # first mps site and first data ket
phi_1_bra = prime(dag(product_state[1])) * mps_bra[1]

ITensor ord=2 (dim=1|id=450|"Link,l=1")' (dim=2|id=316|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}

trace over site 2

In [373]:
s2_tr = mps_ket[2] * noprime(siteinds, mps_bra)[2] 

ITensor ord=4 (dim=2|id=316|"Link,l=1") (dim=2|id=846|"Link,l=2") (dim=2|id=316|"Link,l=1")' (dim=2|id=846|"Link,l=2")'
NDTensors.Dense{Float64, Vector{Float64}}

site 3

In [374]:
s3_tr = mps_ket[3] * noprime(siteinds, mps_bra)[3] 

ITensor ord=4 (dim=2|id=846|"Link,l=2") (dim=2|id=127|"Link,l=3") (dim=2|id=846|"Link,l=2")' (dim=2|id=127|"Link,l=3")'
NDTensors.Dense{Float64, Vector{Float64}}

site 4

In [375]:
s4_tr = mps_ket[4] * noprime(siteinds, mps_bra)[4] 

ITensor ord=4 (dim=2|id=127|"Link,l=3") (dim=2|id=179|"Link,l=4") (dim=2|id=127|"Link,l=3")' (dim=2|id=179|"Link,l=4")'
NDTensors.Dense{Float64, Vector{Float64}}

site 5

In [376]:
s5_tr = mps_ket[5] * noprime(siteinds, mps_bra)[5] 

ITensor ord=2 (dim=2|id=179|"Link,l=4") (dim=2|id=179|"Link,l=4")'
NDTensors.Dense{Float64, Vector{Float64}}

In [377]:
R = s2_tr * s3_tr * s4_tr * s5_tr

ITensor ord=2 (dim=2|id=316|"Link,l=1") (dim=2|id=316|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}

In [378]:
@show R

R = ITensor ord=2
Dim 1: (dim=2|id=316|"Link,l=1")
Dim 2: (dim=2|id=316|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 0.9999999999999997  0.0
 0.0                 0.9999999999999988


ITensor ord=2 (dim=2|id=316|"Link,l=1") (dim=2|id=316|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}

In [379]:
@show phi_1_bra * phi_1_ket * R

phi_1_bra * phi_1_ket * R = ITensor ord=2
Dim 1: (dim=1|id=450|"Link,l=1")'
Dim 2: (dim=1|id=450|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 1×1
 0.5381851435009655


ITensor ord=2 (dim=1|id=450|"Link,l=1")' (dim=1|id=450|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [380]:
site_1_efo = phi_1_bra * phi_1_ket * R

ITensor ord=2 (dim=1|id=450|"Link,l=1")' (dim=1|id=450|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [381]:
@show site_1_efo

site_1_efo = ITensor ord=2
Dim 1: (dim=1|id=450|"Link,l=1")'
Dim 2: (dim=1|id=450|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 1×1
 0.5381851435009655


ITensor ord=2 (dim=1|id=450|"Link,l=1")' (dim=1|id=450|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}

Result should be order 0 but is order 2 because of the way ITensor represents product states as an MPS with bond dimension $\chi = 1$. One way to extract the value is just to use a delta tensor to extract the single component:

In [382]:
delta_tensor = delta(inds(site_1_efo))

ITensor ord=2 (dim=1|id=450|"Link,l=1")' (dim=1|id=450|"Link,l=1")
NDTensors.Diag{Float64, Float64}

In [383]:
@show delta_tensor * site_1_efo

delta_tensor * site_1_efo = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
0.5381851435009655


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [384]:
site_1_overlap = scalar(delta_tensor * site_1_efo)

0.5381851435009655

# Site 2

In [385]:
phi_2_ket = product_state[2] * mps_ket[2] # second mps site and second data ket
phi_2_bra = prime(dag(product_state[2])) * mps_bra[2]

ITensor ord=4 (dim=1|id=450|"Link,l=1")' (dim=1|id=540|"Link,l=2")' (dim=2|id=316|"Link,l=1")' (dim=2|id=846|"Link,l=2")'
NDTensors.Dense{Float64, Vector{Float64}}

trace over site 1

In [386]:
s1_tr = mps_ket[1] * noprime(siteinds, mps_bra)[1] 

ITensor ord=2 (dim=2|id=316|"Link,l=1") (dim=2|id=316|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}

trace over site 3,4,5 already done

In [387]:
L = s1_tr
R = s3_tr * s4_tr * s5_tr

ITensor ord=2 (dim=2|id=846|"Link,l=2") (dim=2|id=846|"Link,l=2")'
NDTensors.Dense{Float64, Vector{Float64}}

In [388]:
site_2_efo = L * phi_2_bra * phi_2_ket * R

ITensor ord=4 (dim=1|id=450|"Link,l=1")' (dim=1|id=540|"Link,l=2")' (dim=1|id=450|"Link,l=1") (dim=1|id=540|"Link,l=2")
NDTensors.Dense{Float64, Vector{Float64}}

In [389]:
@show site_2_efo

site_2_efo = ITensor ord=4
Dim 1: (dim=1|id=450|"Link,l=1")'
Dim 2: (dim=1|id=540|"Link,l=2")'
Dim 3: (dim=1|id=450|"Link,l=1")
Dim 4: (dim=1|id=540|"Link,l=2")
NDTensors.Dense{Float64, Vector{Float64}}
 1×1×1×1
[:, :, 1, 1] =
 0.8262225432485901


ITensor ord=4 (dim=1|id=450|"Link,l=1")' (dim=1|id=540|"Link,l=2")' (dim=1|id=450|"Link,l=1") (dim=1|id=540|"Link,l=2")
NDTensors.Dense{Float64, Vector{Float64}}

In [390]:
delta_tensor = delta(inds(site_2_efo))

ITensor ord=4 (dim=1|id=450|"Link,l=1")' (dim=1|id=540|"Link,l=2")' (dim=1|id=450|"Link,l=1") (dim=1|id=540|"Link,l=2")
NDTensors.Diag{Float64, Float64}

In [391]:
site_2_overlap = scalar(delta_tensor * site_2_efo)
site_2_overlap

0.8262225432485901

# Site 3

In [392]:
phi_3_ket = product_state[3] * mps_ket[3] # second mps site and second data ket
phi_3_bra = prime(dag(product_state[3])) * mps_bra[3]

ITensor ord=4 (dim=1|id=540|"Link,l=2")' (dim=1|id=379|"Link,l=3")' (dim=2|id=846|"Link,l=2")' (dim=2|id=127|"Link,l=3")'
NDTensors.Dense{Float64, Vector{Float64}}

In [393]:
L = s1_tr * s2_tr
R = s4_tr * s5_tr

ITensor ord=2 (dim=2|id=127|"Link,l=3") (dim=2|id=127|"Link,l=3")'
NDTensors.Dense{Float64, Vector{Float64}}

In [394]:
site_3_efo = L * phi_3_ket * phi_3_bra * R

ITensor ord=4 (dim=1|id=540|"Link,l=2") (dim=1|id=379|"Link,l=3") (dim=1|id=540|"Link,l=2")' (dim=1|id=379|"Link,l=3")'
NDTensors.Dense{Float64, Vector{Float64}}

In [395]:
delta_tensor = delta(inds(site_3_efo))

ITensor ord=4 (dim=1|id=540|"Link,l=2") (dim=1|id=379|"Link,l=3") (dim=1|id=540|"Link,l=2")' (dim=1|id=379|"Link,l=3")'
NDTensors.Diag{Float64, Float64}

In [396]:
site_3_overlap = scalar(delta_tensor * site_3_efo)

0.36385701012063354

# Site 4

In [397]:
phi_4_ket = product_state[4] * mps_ket[4] # second mps site and second data ket
phi_4_bra = prime(dag(product_state[4])) * mps_bra[4]

ITensor ord=4 (dim=1|id=379|"Link,l=3")' (dim=1|id=467|"Link,l=4")' (dim=2|id=127|"Link,l=3")' (dim=2|id=179|"Link,l=4")'
NDTensors.Dense{Float64, Vector{Float64}}

In [398]:
L = s1_tr * s2_tr * s3_tr
R = s5_tr

ITensor ord=2 (dim=2|id=179|"Link,l=4") (dim=2|id=179|"Link,l=4")'
NDTensors.Dense{Float64, Vector{Float64}}

In [399]:
site_4_efo = L * phi_4_ket * phi_4_bra * R
delta_tensor = delta(inds(site_4_efo))
site_4_overlap = scalar(delta_tensor * site_4_efo)

0.6319328733379184

# Site 5

In [400]:
phi_5_ket = product_state[5] * mps_ket[5] # second mps site and second data ket
phi_5_bra = prime(dag(product_state[5])) * mps_bra[5]

ITensor ord=2 (dim=1|id=467|"Link,l=4")' (dim=2|id=179|"Link,l=4")'
NDTensors.Dense{Float64, Vector{Float64}}

In [401]:
L = s1_tr * s2_tr * s3_tr * s4_tr


ITensor ord=2 (dim=2|id=179|"Link,l=4") (dim=2|id=179|"Link,l=4")'
NDTensors.Dense{Float64, Vector{Float64}}

In [402]:
site_5_efo = L * phi_5_ket * phi_5_bra * R
delta_tensor = delta(inds(site_5_efo))
site_5_overlap = scalar(delta_tensor * site_5_efo)

0.8472515898392929

# Average 

In [403]:
sum = site_1_overlap + site_2_overlap + site_3_overlap + site_4_overlap + site_5_overlap
sum /= 5

0.6414898320094802

Make into a function

In [429]:
function GetEffectiveOverlap(mps::MPS, product_state::MPS)
    # get ket and bra versions of the mps
    num_sites = length(mps)
    mps_ket = mps # do nothing
    mps_bra = dag(prime(mps))

    # compute traces over each site and store
    traces = Vector{ITensor}(undef, num_sites)
    for s=1:num_sites
        site_trace = mps_ket[s] * noprime(siteinds, mps_bra)[s] # make the site indices match up so we can do the trace
        traces[s] = site_trace
    end

    # now loop over each site and, contract with the data ket for the ket mps and the data bra for the bra mps
    local_overlaps = Vector{Any}(undef, num_sites)
    for s=1:num_sites
        phi_ket = product_state[s] * mps_ket[s] # sth mps site and sth data ket
        phi_bra = prime(dag(product_state[s])) * mps_bra[s] # sth mps site (bra) and sth data bra
        R = 1
        L = 1
        if s == 1
            # first site, only right environment
            for j=(s+1):num_sites
                R *= traces[j]
            end
        elseif s == num_sites
            # last site, only left environment
            for j=1:(s-1)
                L *= traces[j]
            end
        else
            for j=1:(s-1)
                L *= traces[j]
            end
            for j=(s+1):num_sites
                R *= traces[j]
            end
        end
        # contract together
        local_overlap = L * phi_ket * phi_bra * R
        # convert to scalar
        delta_tensor = delta(inds(local_overlap))
        local_overlaps[s] = scalar(delta_tensor * local_overlap)
    end

    sum_val = +(local_overlaps...)
    av_val = sum_val/num_sites


    return local_overlaps, av_val

    #phi_1_ket = product_state[1] * mps_ket[1] # first mps site and first data ket
    #phi_1_bra = prime(dag(product_state[1])) * mps_bra[1]


end

GetEffectiveOverlap (generic function with 1 method)

In [430]:
local_overlaps, av_val = GetEffectiveOverlap(mps, product_state);

In [432]:
av_val

0.6414898320094802

Gives the same result as the manual test. Now let's extend to a large number of sites...

Orthogonalize? 

In [497]:
phi_1 = mps[1] * product_state[1]
phi_1_dag = prime(dag(phi_1))
phi_1_phi_1_dag = phi_1 * phi_1_dag

ITensor ord=4 (dim=2|id=362|"Link,l=1") (dim=1|id=955|"Link,l=1") (dim=2|id=362|"Link,l=1")' (dim=1|id=955|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}

In [498]:
orthogonalize!(mps, 1)

MPS
[1] ((dim=2|id=540|"S=1/2,Site,n=1"), (dim=2|id=362|"Link,l=1"))
[2] ((dim=2|id=362|"Link,l=1"), (dim=2|id=615|"S=1/2,Site,n=2"), (dim=2|id=585|"Link,l=2"))
[3] ((dim=2|id=585|"Link,l=2"), (dim=2|id=927|"S=1/2,Site,n=3"), (dim=2|id=78|"Link,l=3"))
[4] ((dim=2|id=78|"Link,l=3"), (dim=2|id=814|"S=1/2,Site,n=4"), (dim=2|id=759|"Link,l=4"))
[5] ((dim=2|id=759|"Link,l=4"), (dim=2|id=945|"S=1/2,Site,n=5"))


Use a delta tensor to contract over the primed and regular site indices

In [499]:
delta(sites[2], sites[2]')

ITensor ord=2 (dim=2|id=615|"S=1/2,Site,n=2") (dim=2|id=615|"S=1/2,Site,n=2")'
NDTensors.Diag{Float64, Float64}

In [500]:
s2_tr = mps[2] * prime(dag(mps[2])) * delta(sites[2], sites[2]')

ITensor ord=4 (dim=2|id=362|"Link,l=1") (dim=2|id=585|"Link,l=2") (dim=2|id=362|"Link,l=1")' (dim=2|id=585|"Link,l=2")'
NDTensors.Dense{Float64, Vector{Float64}}

In [501]:
s3_tr = mps[3] * prime(dag(mps[3])) * delta(sites[3], sites[3]')

ITensor ord=4 (dim=2|id=585|"Link,l=2") (dim=2|id=78|"Link,l=3") (dim=2|id=585|"Link,l=2")' (dim=2|id=78|"Link,l=3")'
NDTensors.Dense{Float64, Vector{Float64}}

In [503]:
s4_tr = mps[4] * prime(dag(mps[4])) * delta(sites[4], sites[4]')

ITensor ord=4 (dim=2|id=78|"Link,l=3") (dim=2|id=759|"Link,l=4") (dim=2|id=78|"Link,l=3")' (dim=2|id=759|"Link,l=4")'
NDTensors.Dense{Float64, Vector{Float64}}

In [504]:
s5_tr = mps[5] * prime(dag(mps[5])) * delta(sites[5], sites[5]')

ITensor ord=2 (dim=2|id=759|"Link,l=4") (dim=2|id=759|"Link,l=4")'
NDTensors.Dense{Float64, Vector{Float64}}

Contract the traces

In [507]:
contracted_traces = s2_tr * s3_tr * s4_tr * s5_tr

ITensor ord=2 (dim=2|id=362|"Link,l=1") (dim=2|id=362|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}

In [508]:
@show contracted_traces

contracted_traces = ITensor ord=2
Dim 1: (dim=2|id=362|"Link,l=1")
Dim 2: (dim=2|id=362|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 0.9999999999999997      2.7755575615628914e-17
 2.7755575615628914e-17  0.9999999999999988


ITensor ord=2 (dim=2|id=362|"Link,l=1") (dim=2|id=362|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}

In [512]:
@show phi_1_phi_1_dag * contracted_traces

phi_1_phi_1_dag * contracted_traces = ITensor ord=2
Dim 1: (dim=1|id=955|"Link,l=1")
Dim 2: (dim=1|id=955|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}
 1×1
 0.5381851435009655


ITensor ord=2 (dim=1|id=955|"Link,l=1") (dim=1|id=955|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}